In [1]:
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
from cicada import transcript

handler = logging.StreamHandler()
handler.setFormatter(transcript.Formatter())
transcript.logger.addHandler(handler)

In [3]:
import numpy

from cicada.additive import AdditiveProtocolSuite
from cicada.communicator import SocketCommunicator

def main(communicator):
    transcript.log(transcript.Category.APP, "Let's share some secrets!", comm=communicator.name, rank=communicator.rank)
    protocol = AdditiveProtocolSuite(communicator)
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log(transcript.Category.APP, "Let's add some secrets!", comm=communicator.name, rank=communicator.rank)
    c_share = protocol.add(a_share, b_share)
    transcript.log(transcript.Category.APP, "Let's reveal the results!", comm=communicator.name, rank=communicator.rank)
    c = protocol.reveal(c_share)

with transcript.record():
    SocketCommunicator.run(fn=main, world_size=3);

Player 0 Let's share some secrets!
Player 1 Let's share some secrets!
Player 2 Let's share some secrets!
Player 0 PRZSProtocol.__init__
Player 1 PRZSProtocol.__init__
Player 2 PRZSProtocol.__init__
Player 0 --> 1 PRZS 6836353792030665433
Player 1 --> 2 PRZS 2002159558738414436
Player 2 --> 0 PRZS 4246968382239603197
Player 0 <-- 2 PRZS 4246968382239603197
Player 1 <-- 0 PRZS 6836353792030665433
Player 2 <-- 1 PRZS 2002159558738414436
Player 0 AdditiveProtocolSuite.share
Player 1 AdditiveProtocolSuite.share
Player 2 AdditiveProtocolSuite.share
Player 0 FixedPoint.encode [] 131072
Player 1 AdditiveProtocolSuite.share
Player 2 AdditiveProtocolSuite.share
Player 0 Field.inplace_add [75933736736951368, 131072] None
Player 1 FixedPoint.encode [] 196608
Player 0 AdditiveProtocolSuite.share
Player 1 Field.inplace_add [16353075924711440279, 196608] None
Player 2 Let's add some secrets!
Player 0 Let's add some secrets!
Player 1 Let's add some secrets!
Player 2 Field.add [18349144187934451992, 24